In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

import numpy as np
from functools import partial

In [ ]:
import sys, os
if os.path.abspath('../') not in sys.path:
    sys.path.append(os.path.abspath('../'))

In [ ]:
from maxvol_compression.sketch_matrix import FastFrequentDirections, RandomSums

In [ ]:
BATCH_SIZE = 10

In [ ]:
class DummyDatasetCifar10:
    def __init__(self, batch_size=4, data_root='./data'):
        self.transform = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        self.batch_size = batch_size
        self.trainset = torchvision.datasets.CIFAR10(root=data_root, 
                                                     train=True,
                                                     download=True, 
                                                     transform=self.transform)
        self.trainloader = torch.utils.data.DataLoader(self.trainset, 
                                                       batch_size=self.batch_size,
                                                       shuffle=True, 
                                                       drop_last=True,
                                                       num_workers=2)
        self.testset = torchvision.datasets.CIFAR10(root=data_root, 
                                                    train=False,
                                                    download=True, 
                                                    transform=self.transform)
        self.testloader = torch.utils.data.DataLoader(self.testset, 
                                                      batch_size=self.batch_size,
                                                      shuffle=False, 
                                                      drop_last=True,
                                                      num_workers=2)
        self.classes = np.array(['plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck'])

In [ ]:
class DummyModelCifar10(nn.Module):
    def __init__(self):
        super().__init__()
        self.num_classes = 10
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, self.num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
model = DummyModelCifar10()
cifar10 = DummyDatasetCifar10(batch_size=BATCH_SIZE, data_root='../data')

In [ ]:
model.eval()

In [ ]:
# Test run to compute bounds
fds = FastFrequentDirections(150, 400, keep_original=True)
def update_sketchmatrix(self, input, output, alg):
    batch_size = input[0].shape[0]
    alg.update(input[0].view(batch_size, -1).cpu().numpy())
    
handle = model.fc1.register_forward_hook(
    partial(update_sketchmatrix, alg=fds))

In [ ]:
# handle.remove()

In [ ]:
%%time
for i, (batch,_) in enumerate(cifar10.testloader, 1):
    with torch.no_grad():
        _ = model(batch)
    if i % 200 == 0: print(f'{i} batches completed')
handle.remove()

In [ ]:
fds.sketch_matrix[:10]

In [ ]:
fds.compute_error()

In [ ]:
# Test run to compute bounds
rs = RandomSums(150, 400, keep_original=True)
def update_sketchmatrix(self, input, output, alg):
    batch_size = input[0].shape[0]
    alg.update(input[0].view(batch_size, -1).cpu().numpy())
    
handle = model.fc1.register_forward_hook(
    partial(update_sketchmatrix, alg=rs))

In [ ]:
%%time
for i, (batch,_) in enumerate(cifar10.testloader, 1):
    with torch.no_grad():
        _ = model(batch)
    if i % 200 == 0: print(f'{i} batches completed')
handle.remove()

In [ ]:
rs.sketch_matrix[:10]

In [ ]:
rs.compute_error()